# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Jun  3 2022  2:34PM  238796        19  ACG-2102490232   
1  Jun  3 2022  2:31PM  238795        18  INNOVA-8099538   
2  Jun  3 2022  2:09PM  238792        10      0085739953   
3  Jun  3 2022  2:09PM  238792        10      0085739996   
4  Jun  3 2022  2:09PM  238792        10      0085739934   
5  Jun  3 2022  2:09PM  238792        10      0085739931   
6  Jun  3 2022  2:09PM  238792        10      0085739933   
7  Jun  3 2022  2:09PM  238792        10      0085739930   
8  Jun  3 2022  2:09PM  238792        10      0085739929   
9  Jun  3 2022  2:09PM  238792        10      0085739943   

                Customer ShipmentStatus  
0  ACG North America LLC       Released  
1     Innova Softgel LLC       Released  
2      ISDIN Corporation       Released  
3      ISDIN Corporation       Released  
4      ISDIN Corporation       Released  
5      ISDIN Corporation       Released  
6      ISDIN Corporation       Released  
7      ISDIN Corporation       Released  
8      ISDIN Corporation       Released  
9      ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
30  238791       Released          1
31  238792       Released         44
32  238793       Released          1
33  238795       Released          1
34  238796       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
238791                NaN        NaN       1.0
238792                NaN        NaN      44.0
238793                NaN        NaN       1.0
238795                NaN        NaN       1.0
238796                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238711                0.0        0.0       1.0
238713                9.0        8.0       1.0
238720                0.0        0.0       1.0
238721                0.0        0.0       1.0
238748                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
238711                  0          0         1
238713                  9          8         1
238720                  0          0         1
238721                  0          0         1
238748                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               238711          0          0         1
1               238713          9          8         1
2               238720          0          0         1
3               238721          0          0         1
4               238748          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               238711                             1
1               238713         9         8         1
2               238720                             1
3               238721                             1
4               238748                             1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Jun  3 2022  2:34PM  238796        19  ACG North America LLC
1   Jun  3 2022  2:31PM  238795        18     Innova Softgel LLC
2   Jun  3 2022  2:09PM  238792        10      ISDIN Corporation
46  Jun  3 2022  2:06PM  238783        10      ISDIN Corporation
91  Jun  3 2022  2:05PM  238793        10      ISDIN Corporation
92  Jun  3 2022  2:01PM  238791        18         ClearSpec, LLC
93  Jun  3 2022  1:59PM  238790        18         ClearSpec, LLC
94  Jun  3 2022  1:51PM  238789        18         ClearSpec, LLC
95  Jun  3 2022  1:45PM  238788        19  ACG North America LLC
96  Jun  3 2022  1:40PM  238787        16        SHL Pharma, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Jun  3 2022  2:34PM  238796        19  ACG North America LLC             
1  Jun  3 2022  2:31PM  238795        18     Innova Softgel LLC             
2  Jun  3 2022  2:09PM  238792        10      ISDIN Corporation             
3  Jun  3 2022  2:06PM  238783        10      ISDIN Corporation             
4  Jun  3 2022  2:05PM  238793        10      ISDIN Corporation             
5  Jun  3 2022  2:01PM  238791        18         ClearSpec, LLC             
6  Jun  3 2022  1:59PM  238790        18         ClearSpec, LLC             
7  Jun  3 2022  1:51PM  238789        18         ClearSpec, LLC             
8  Jun  3 2022  1:45PM  238788        19  ACG North America LLC             
9  Jun  3 2022  1:40PM  238787        16        SHL Pharma, LLC             

  Executing  Released  
0                   1  
1                   1  
2                  44  
3                  45  
4                   1  
5                   1  
6                   1  
7                   1  
8                   1  
9                   1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer  Released  \
0  Jun  3 2022  2:34PM  238796        19  ACG North America LLC         1   
1  Jun  3 2022  2:31PM  238795        18     Innova Softgel LLC         1   
2  Jun  3 2022  2:09PM  238792        10      ISDIN Corporation        44   
3  Jun  3 2022  2:06PM  238783        10      ISDIN Corporation        45   
4  Jun  3 2022  2:05PM  238793        10      ISDIN Corporation         1   
5  Jun  3 2022  2:01PM  238791        18         ClearSpec, LLC         1   
6  Jun  3 2022  1:59PM  238790        18         ClearSpec, LLC         1   
7  Jun  3 2022  1:51PM  238789        18         ClearSpec, LLC         1   
8  Jun  3 2022  1:45PM  238788        19  ACG North America LLC         1   
9  Jun  3 2022  1:40PM  238787        16        SHL Pharma, LLC         1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jun  3 2022  2:34PM  238796        19  ACG North America LLC         1   
1  Jun  3 2022  2:31PM  238795        18     Innova Softgel LLC         1   
2  Jun  3 2022  2:09PM  238792        10      ISDIN Corporation        44   
3  Jun  3 2022  2:06PM  238783        10      ISDIN Corporation        45   
4  Jun  3 2022  2:05PM  238793        10      ISDIN Corporation         1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jun  3 2022  2:34PM  238796        19  ACG North America LLC         1   
1  Jun  3 2022  2:31PM  238795        18     Innova Softgel LLC         1   
2  Jun  3 2022  2:09PM  238792        10      ISDIN Corporation        44   
3  Jun  3 2022  2:06PM  238783        10      ISDIN Corporation        45   
4  Jun  3 2022  2:05PM  238793        10      ISDIN Corporation         1   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jun  3 2022  2:34PM  238796        19  ACG North America LLC         1   
1  Jun  3 2022  2:31PM  238795        18     Innova Softgel LLC         1   
2  Jun  3 2022  2:09PM  238792        10      ISDIN Corporation        44   
3  Jun  3 2022  2:06PM  238783        10      ISDIN Corporation        45   
4  Jun  3 2022  2:05PM  238793        10      ISDIN Corporation         1   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2387618       145       10.0        9.0
102         477554         2        0.0        0.0
12          477541         2        0.0        0.0
15          238755         1        0.0        0.0
16          477535         2        0.0        0.0
18         1432606         6        0.0        0.0
19          716358         4        0.0        0.0
20          477549        28       23.0        0.0
22          477559         2        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2387618       145       10.0        9.0
1       102   477554         2        0.0        0.0
2        12   477541         2        0.0        0.0
3        15   238755         1        0.0        0.0
4        16   477535         2        0.0        0.0
5        18  1432606         6        0.0        0.0
6        19   716358         4        0.0        0.0
7        20   477549        28       23.0        0.0
8        22   477559         2        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10       145       10.0        9.0
1       102         2        0.0        0.0
2        12         2        0.0        0.0
3        15         1        0.0        0.0
4        16         2        0.0        0.0
5        18         6        0.0        0.0
6        19         4        0.0        0.0
7        20        28       23.0        0.0
8        22         2        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  145.0
1       102  Released    2.0
2        12  Released    2.0
3        15  Released    1.0
4        16  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10  102   12   15   16   18   19    20   22
Status                                                   
Completed    9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
Executing   10.0  0.0  0.0  0.0  0.0  0.0  0.0  23.0  0.0
Released   145.0  2.0  2.0  1.0  2.0  6.0  4.0  28.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10  102   12   15   16   18   19    20   22
0          Completed    9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
1          Executing   10.0  0.0  0.0  0.0  0.0  0.0  0.0  23.0  0.0
2           Released  145.0  2.0  2.0  1.0  2.0  6.0  4.0  28.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10  102   12   15   16   18   19    20   22
0  Completed    9.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0
1  Executing   10.0  0.0  0.0  0.0  0.0  0.0  0.0  23.0  0.0
2   Released  145.0  2.0  2.0  1.0  2.0  6.0  4.0  28.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()